In [1]:
import numpy as np
import pandas as pd
import collections
#from textblob import TextBlob as tb
import nltk
from time import time

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/italohmb/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Creating constants that will be used over this report

In [3]:
COLUMN_AXIS = 1
FULL_REPORT_COLNAME = 'noticia'
CONTENT_COLNAME = 'conteudo'
SUBTITLE_COLNAME = 'subTitulo'
TITLE_COLNAME = 'titulo'
TOKENS_COLNAME = 'tokens'
TERM_COLNAME = 'term'
REPORT_ID_COLNAME = 'idNoticia'
AND = 'AND'
OR = 'OR'

# Meeting data

In [4]:
df = pd.read_csv('../../data/estadao_noticias_eleicao.csv')

In [5]:
df.head()

,timestamp,titulo,subTitulo,conteudo,url,idNoticia
0,2014-12-31T00:00:00Z,PT espera 30 mil pessoas em festa na Esplanada,Objetivo é demonstrar apoio popular a Dilma e ...,BRASÍLIA - Após o desgaste provocado com o lan...,"http://politica.estadao.com.br/noticias/geral,...",1
1,2014-12-31T00:00:00Z,Alckmin toma posse de olho no Planalto,Governador reeleito tenta amarrar tucanos paul...,"Reeleito em outubro, o governador tucano Geral...","http://politica.estadao.com.br/noticias/geral,...",2
2,2014-12-31T00:00:00Z,Seis obstáculos e desafios do segundo mandato ...,"Em meio a escândalo de corrupção, presidente t...",1. Rearranjo das contas A nova equipe econôm...,"http://politica.estadao.com.br/noticias/geral,...",3
3,2014-12-31T00:00:00Z,NaN,Veja as principais fotos do dia e dos eventos ...,NaN,"http://fotos.estadao.com.br/fotos/politica,dil...",4
4,2014-12-31T00:00:00Z,NaN,Veja as principais fotos do dia e dos eventos ...,NaN,"http://fotos.estadao.com.br/fotos/politica,dil...",5


In [6]:
print("Data has %d rows and %d columns" % df.shape)

Data has 8716 rows and 6 columns


# Concatenatin galls reports' title and content in just one column.

In [7]:
def concatenate_report(row):
    """Concatenate report title and content in just one column.
        
    Args:
        row (:obj: pandas.Series): one row observation from a pandas.DataFrame.            

    Return:
        str: full report (content with title) in lowercase.
    """
    title = str(row[TITLE_COLNAME])
    subtitle = str(row[SUBTITLE_COLNAME])
    content = str(row[CONTENT_COLNAME])
    full_report = title + " " + subtitle + " " + content
    return full_report.lower()

Replacing content values that are NaN (not a number) for an empty string.

In [8]:
empty_str = ""
df[TITLE_COLNAME].fillna(empty_str, inplace=True)
df[SUBTITLE_COLNAME].fillna(empty_str, inplace=True)
df[CONTENT_COLNAME].fillna(empty_str, inplace=True)

In [9]:
df[FULL_REPORT_COLNAME] = df.apply(
    lambda row: concatenate_report(row), axis=COLUMN_AXIS)

Selecting just report's id and full content columns:

In [10]:
df = df[[REPORT_ID_COLNAME, FULL_REPORT_COLNAME]]

Dataframe now looks like:

In [11]:
df.head()

,idNoticia,noticia
0,1,pt espera 30 mil pessoas em festa na esplanada...
1,2,alckmin toma posse de olho no planalto governa...
2,3,seis obstáculos e desafios do segundo mandato ...
3,4,veja as principais fotos do dia e dos eventos...
4,5,veja as principais fotos do dia e dos eventos...


# Tokenizing report's text and saving tokens in another column in dataframe

In [12]:
def tokenize_text(row):
    """Tokenize the text content of a report given as a row from a DataFrame
        
    Args:
        row (:obj: pandas.Series): one row observation from a pandas.DataFrame.            

    Return:
        set: a report content turned into a set of tokens.
    """    
    
    #text_blob = tb(row[FULL_REPORT_COLNAME]) 
    #m_tokens = set(text_blob.words)
    m_tokens = nltk.word_tokenize(row[FULL_REPORT_COLNAME])
    return m_tokens

In [13]:
df[TOKENS_COLNAME] = df.apply(
    lambda row: tokenize_text(row), axis=COLUMN_AXIS)

# Creating inverted index

First, we will create a intermediate structure called unnested_tokens. This structure will save each token of a report individually, associating it to the report's id. After this step, we will group this unnested_tokens structure by tokens, getting all reports' ids where one specific token appears.

In [14]:
def unnest_tokens_report(unnested_tokens_list, row):
    """Given a row observation of a DataFrame to represent a report (with content,
    tokens and id), iterate over the set of tokens and save each one as a dict with 
    token value and report id. Each dict is appended in the unnested_tokens_list
    passed as param.
        
    Args:
        unnested_tokens_list (list): list of dicts, each dict containing a token value 
            and the report id where it occured.
        row (:obj: pandas.Series): one row observation from a pandas.DataFrame.            
    """  
    
    for token in row[TOKENS_COLNAME]:
        new_row = {
            TERM_COLNAME: token.strip('\'').strip(),
            REPORT_ID_COLNAME: row[REPORT_ID_COLNAME]}
        unnested_tokens_list.append(new_row)

In [15]:
unnested_tokens_list = []
df.apply(lambda row: unnest_tokens_report(unnested_tokens_list, row), axis=COLUMN_AXIS)

print("\nThe unnested_tokens_list looks like: \n")
print(unnested_tokens_list[:8])
print("\nThe 'list of dicts' format will be used to create a pandas.DataFrame:")


The unnested_tokens_list looks like: 

[{'term': 'pt', 'idNoticia': 1}, {'term': 'espera', 'idNoticia': 1}, {'term': '30', 'idNoticia': 1}, {'term': 'mil', 'idNoticia': 1}, {'term': 'pessoas', 'idNoticia': 1}, {'term': 'em', 'idNoticia': 1}, {'term': 'festa', 'idNoticia': 1}, {'term': 'na', 'idNoticia': 1}]

The 'list of dicts' format will be used to create a pandas.DataFrame:


In [16]:
unnested_tokens_df = pd.DataFrame(unnested_tokens_list)
unnested_tokens_df.head(10)

,idNoticia,term
0,1,pt
1,1,espera
2,1,30
3,1,mil
4,1,pessoas
5,1,em
6,1,festa
7,1,na
8,1,esplanada
9,1,objetivo


### Grouping by term to create inverted index

In [17]:
class InvertedIndexTerm:
    """Class for register term frequency and docs' ids in which a 
    term of a inverted index structure appears.
    
    Attributes:
        term_freq (int): Quantity of docs in which term appears.
        docs_ids (list): ids of docs in which term appears.
    """
    
    def __init__(self, term, freq, docs_ids):
        self.term = term
        self.freq = freq
        self.docs_ids = sorted(list(docs_ids))
    
    def get_term(self):
        return self.term
    
    def get_freq(self):
        return self.freq
    
    def get_docs_ids(self):
        return self.docs_ids

In [18]:
def create_inverted_index_structure(unnested_tokens_df):
    """Create a inverted index structure using python dict structure.
    
    Args:
        unnested_tokens_df (:obj: pandas.DataFrame): unnested tokens of texts from a set of texts.
    
    Return:
        dict: keys are terms found at texts and values are lists of docs_ids where terms are found.
    """    
    inverted_index = dict()

    for term, group_itens in unnested_tokens_df.groupby([TERM_COLNAME]):

        term_freq = len(group_itens.get_values())
        docs_ids = set(group_itens[REPORT_ID_COLNAME])
        inverted_index[term] = InvertedIndexTerm(term, term_freq, docs_ids)
    
    return inverted_index

Creating the inverted index structure for our data:

In [19]:
m_inverted_index = create_inverted_index_structure(unnested_tokens_df)

## Processing Queries

In [20]:
def is_one_term_query(query):
    
    empty_str, space_str = "", " "    
    
    if query == empty_str or query == space_str:
        raise ValueError('You should search for a non empty string.')        
    else:
        return len(query.split(space_str)) == 1

In [21]:
def get_query_operator(query):    
    """Get boolean query operator.
    
    Args:
        query (str): query with more than one term.
    
    Return:
        str: string name for the boolean query operator.
    """  
    
    if AND not in query and OR not in query:
        raise ValueError('Thats not a valid query.')  
    
    return AND if AND in query else OR

In [22]:
def lowercase_iterable_itens(iterable):
    """Lowercase all itens in a iterable object.
    
    Args:
        iterable (list): list of str itens to lowercase.
    
    Return:
        list: list with elements in lowercase.
    """      
    return list(map(lambda term: term.lower(), iterable))

In [23]:
def _intersect(list1, list2):
    """Found all elements that are commom to two lists."""           
    
    result = []    
    i, j = 0, 0    
    
    while i < len(list1) and j < len(list2):    
        if list1[i] == list2[j]:
            result.append(list1[i])
            i += 1
            j += 1        
        else:            
            if list1[i] < list2[j]:
                i += 1
            else:
                j += 1
    
    return result

In [24]:
def _union(list1, list2):
    """Found the union (with no repetition) of elements of two given lists."""           
    
    result = []    
    i, j = 0, 0
    
    while i < len(list1) and j < len(list2):    
        if list1[i] < list2[j]:
            result.append(list1[i])
            i += 1      
        elif list1[i] > list2[j]:            
            result.append(list2[j])
            j += 1 
        else:
            result.append(list1[i])
            i += 1
            j += 1

    while i < len(list1):    
        result.append(list1[i])
        i += 1      
    
    while j < len(list2):    
        result.append(list2[j])
        j += 1      
            
    return result

In [25]:
def sort_terms_list_per_freq(terms_list, inverted_index):
    """Sort list of terms to search by their frequency. Frequency of a term can be found in the InvertedIndexTerm
    object saved in the inverted index structure to the corresponding key term.
    
    Args:
        terms_list (list): list to sort.
        inverted_index (dict): structure through which get term's frequency.
    
    Return:
        list: sorted list of InvertedIndexTerm objects.
    """      
    terms_obj_list = list(map(lambda term: inverted_index[term], terms_list))    
    terms_obj_list.sort(key= lambda term: term.get_freq())
    return terms_obj_list

In [26]:
def _boolean_search(terms_to_search, operator, inverted_index):
      
    sorted_terms_per_freq = sort_terms_list_per_freq(terms_to_search, inverted_index)
    
    docs_ids = sorted_terms_per_freq[0].get_docs_ids()   
    result = docs_ids
    
    for another_term in sorted_terms_per_freq:
        
        docs_ids = another_term.get_docs_ids()  
    
        if operator == AND:
            result = _intersect(result, docs_ids)
        elif operator == OR:
            result = _union(result, docs_ids)
            
    return list(result)

In [27]:
def search(query, inverted_index):
    
    if is_one_term_query(query):
        term = query.lower()
        return inverted_index[term].get_docs_ids()  
    
    else:    
        operator = get_query_operator(query)
        terms_to_search = query.split(" " + operator + " ")
        terms_to_search = lowercase_iterable_itens(terms_to_search)
        
        return _boolean_search(terms_to_search, operator, inverted_index)

# PARTE DOIS

### Anotações do lab

Na representação binária eu que escolho o critério de ranking entre os documentos. O ranking pode ser: 
- random
- precission / recall

Na verdade a busca binária só vai ser usada pelos outros modelos, já que ela própria não oferece ranking. Os outros modelos usam os resultados da busca binária para poder criar um ranking.

In [28]:
# a decisão de salvar a frequência do documennto em uma estrutura fora do inverted_index
# é que por definição o inverted_index só contem a chave termo e o valor docs onde ele aparece e 
# a frequência do termo em todos os docs

def calc_doc_terms_frequency(doc_id, doc_tokens, inverted_index, big_term_freq_dict):
    tokens_frequencies = collections.Counter(doc_tokens)
    big_term_freq_dict[doc_id] = tokens_frequencies

df.apply(
    lambda row: calc_doc_terms_frequency(
        row[REPORT_ID_COLNAME], row[TOKENS_COLNAME], m_inverted_index, big_term_freq_dict), 
    axis=COLUMN_AXIS)

print("")

In [68]:
class TokenEstimator:
    
    def __init__(self, df, inverted_index):
        self.inverted_index = inverted_index
        self.big_term_freq_dict = dict()
        self._calc_terms_frequency()
        self.df = df
    
    def _calc_doc_terms_frequency(self, doc_id, doc_tokens):
        tokens_frequencies = collections.Counter(doc_tokens)
        self.big_term_freq_dict[doc_id] = tokens_frequencies
    
    def _calc_terms_frequency(self):
        df.apply(
            lambda row: self._calc_doc_terms_frequency(
                row[REPORT_ID_COLNAME], row[TOKENS_COLNAME]), 
            axis=COLUMN_AXIS)
    
    def get_tf(self, term, doc_id):
        if doc_id not in self.big_term_freq_dict:
            raise ValueError('There is no such doc_id in the set of docs.')
        return self.big_term_freq_dict[doc_id][term]
    
    def calc_idf(self, term):
        NUMBER_OF_ROWS_INDEX = 0
        n_documents = self.df.shape[NUMBER_OF_ROWS_INDEX]
        n_containing_term = len(self.inverted_index[term].get_docs_ids())
        idf = np.log((n_documents +1) / (n_containing_term))
        return idf
    
    def calc_tfidf(self, term, doc_id):
        tf = self.get_tf(term, doc_id)
        idf = self.calc_idf(term)
        return tf * idf
        
#    def sim(self, query, doc)        
        
#        inverted_index_term = self.inverted_index[term]        
#        term_docs = term.get_docs_ids()        

In [78]:
token_estimator = TokenEstimator(df, m_inverted_index)

In [125]:
class Scorer(object):
    def __init__(self, token_estimator):
        self.token_estimator = token_estimator

class Binary(Scorer):
    def __init__(self):
        Scorer.__init_(self)
        
class TermFrequency(Scorer):

    def __init__(self, token_estimator):
        Scorer.__init_(self, token_estimator)
    
    def ranking_search(self, query, search_result):
        ranking = []
        query_terms = query.split(" AND ")
        for doc_id in search_result:
            tf = sim(query_terms, doc_id)
            ranking.append((doc_id, tf))
        ranking = sorted(ranking, key=lambda t: t[1], reverse=True)
        return ranking
    
    def sim(self, query_terms, doc_id):
        tf_accumulated = 0
        for query_term in query_terms:
            tf = Scorer.token_estimator.get_tf(query_term, doc_id)
            tf_accumulated += tf
        ranking.append((doc_id, tf_accumulated))        

class Idf(Scorer):
    def __init__(self):
        Scorer.__init__(self)    
    
class BM25(Scorer):
    def __init__(self):
        Scorer.__init__(self)

In [122]:
# query = "segundo AND turno"
# merged_docs_id = search(query, m_inverted_index)

In [ ]:
# ranking = []
# query_terms = query.split(" AND ")
# for doc_id in merged_docs_id:
#     tf_accumulated = 0
#     for query_term in query_terms:
#         tf = token_estimator.get_tf(query_term, doc_id)
#         tf_accumulated += tf
#     ranking.append((doc_id, tf_accumulated))

# sorted(ranking, key=lambda t: t[1], reverse=True)

TODO: 
- unificar token e term para token
- idf: np.log(n_documents / (1 + self.n_containing_term(term)))
- tf: freq_term / sum_all_freq_in_doc